In [4]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import LLMChain, APIChain
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.schema import Document
from langchain.tools import BaseTool
from langchain.agents import ZeroShotAgent, AgentExecutor, AgentType, initialize_agent, Tool, load_tools
from langchain.agents.agent_toolkits.openapi import planner
from langchain.requests import TextRequestsWrapper
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.api import open_meteo_docs
from datetime import datetime, timedelta
import requests
import pandas as pd
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

search_endpoint = 'https://search-service01.search.windows.net'
search_key = '73Swa5YqUR5IRMwUIqOH6ww2YBm3SveLv7rDmZVXtIAzSeBjEQe9'
private_index_name = "nois-private-v3-index"

retriever_private = SearchClient(
    endpoint=search_endpoint,
    index_name=private_index_name,
    credential=AzureKeyCredential(search_key),
    b=0.0,
    k1=0.3,
    searchMode="all"
)

In [5]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-3.5-turbo")


def count_tokens(s):
    return len(enc.encode(s))
string = '''[{'id': '262aa143-307a-4b9f-848d-1e6bf2e5cacb', 'name': 'Hung Bui', 'email': 'hung.bui@nois.vn', 'fullName': 'BÙI TUẤN HƯNG', 'jobTitle': 'Lập trình viên', 'mobilePhone': '0966772170', 'birthday': '1997-08-17T00:00:00', 'offDay': 0.5, 'yearOffDay': 6, 'maxDayOff': 15, 'remainDayOffLastYear': 0, 'offDayUseRamainDayOffLastYear': 1, 'gender': 1, 'personalEmail': 'hungbt.dev@gmail.com', 'avatar': 'https://noishrm.blob.core.windows.net/avatar/8fec9283-e82f-43ec-8f61-addb732dabcb.jpeg', 'acceptOfferDate': '2022-10-31T00:00:00', 'quitJobDate': '', 'bonusDayOff': 0, 'managerId': '', 'managerFullName': '', 'departmentId': 0, 'departmentName': 'Development Unit 1', 'groupUserId': '', 'groupUserName': 'Nhân viên chính thức', 'hasLeft': '', 'identityCard': '', 'dateOfIssue': '2021-08-10T00:00:00', 'issuedBy': 'Cục trưởng CCS QLHC về TTXH', 'placeOfTemporaryResidence': '502/15/2 Thống Nhất, p.16, Gò Vấp, TP.HCM', 'placeOfResidence': '161 Đường 458, Ấp Hội Thạnh, Xã Trung An, Huyện Củ Chi, TP.HCM', 'relativeName': 'Lê Thị Kiều', 'relationship': 'Mẹ', 'relativeMobilePhone': '', 'hasBankAccount': '', 'bankName': 'Vietcombank', 'bankBranch': 'DONG SAI GON-PGD THANH DA', 'bankAccountNumber': '0531002579897', 'birthPlace': 'Ấp Hội Thạnh, Xã Trung An, Huyện Củ Chi, TP.HCM', 'isFirstLogin': '', 'roles': '', 'taxIdentificationNumber': '8460719408', 'education': '', 'jobId': '', 'jobName': 'Developer', 'levelId': '', 'levelName': ''}, {'id': '72fb1693-9cc1-4923-b0a4-afb18469613c', 'name': 'KHASNH BUIF', 'email': '', 'fullName': '', 'jobTitle': '', 'mobilePhone': '', 'birthday': '', 'offDay': 1, 'yearOffDay': 0, 'maxDayOff': 0, 'remainDayOffLastYear': 0, 'offDayUseRamainDayOffLastYear': 0, 'gender': 0, 'personalEmail': '', 'avatar': '', 'acceptOfferDate': '', 'quitJobDate': '', 'bonusDayOff': 0, 'managerId': '', 'managerFullName': '', 'departmentId': 0, 'departmentName': '', 'groupUserId': '', 'groupUserName': '', 'hasLeft': '', 'identityCard': '', 'dateOfIssue': '', 'issuedBy': '', 'placeOfTemporaryResidence': '', 'placeOfResidence': '', 'relativeName': '', 'relationship': '', 'relativeMobilePhone': '', 'hasBankAccount': '', 'bankName': '', 'bankBranch': '', 'bankAccountNumber': '', 'birthPlace': '', 'isFirstLogin': '', 'roles': '', 'taxIdentificationNumber': '', 'education': '', 'jobId': '', 'jobName': '', 'levelId': '', 'levelName': ''}, {'id': 'c8604118-7db6-4e08-8926-5bd8c81bce81', 'name': 'Bao Ho', 'email': 'honguyenngocbao@gmail.com', 'fullName': 'HỒ NGUYỄN NGỌC BẢO', 'jobTitle': '', 'mobilePhone': '', 'birthday': '', 'offDay': 0, 'yearOffDay': 0, 'maxDayOff': 0, 'remainDayOffLastYear': 0, 'offDayUseRamainDayOffLastYear': 0, 'gender': 0, 'personalEmail': '', 'avatar': '', 'acceptOfferDate': '', 'quitJobDate': '', 'bonusDayOff': 0, 'managerId': '', 'managerFullName': '', 'departmentId': 0, 'departmentName': '', 'groupUserId': '', 'groupUserName': '', 'hasLeft': '', 'identityCard': '', 'dateOfIssue': '', 'issuedBy': '', 'placeOfTemporaryResidence': '', 'placeOfResidence': '', 'relativeName': '', 'relationship': '', 'relativeMobilePhone': '', 'hasBankAccount': '', 'bankName': '', 'bankBranch': '', 'bankAccountNumber': '', 'birthPlace': '', 'isFirstLogin': '', 'roles': '', 'taxIdentificationNumber': '', 'education': '', 'jobId': '', 'jobName': '', 'levelId': '', 'levelName': ''}]'''

print(count_tokens(string))

1171


In [95]:
# Load up your LLM
llm = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_base='https://openai-nois-intern.openai.azure.com/',
    openai_api_version="2023-03-15-preview",
    deployment_name='gpt-35-turbo',
    openai_api_key='400568d9a16740b88aff437480544a39',
    temperature=0.5,
    max_tokens=450
)

llm2 = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_base='https://openai-nois-intern.openai.azure.com/',
    openai_api_version="2023-03-15-preview",
    deployment_name='gpt-35-turbo',
    openai_api_key='400568d9a16740b88aff437480544a39',
    temperature=0,
    max_tokens=600,
    stop=['\n', '<|im_end|>', '<|im_sep|>']
)

llm3 = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_base='https://openai-nois-intern.openai.azure.com/',
    openai_api_version="2023-03-15-preview",
    deployment_name='gpt-35-turbo',
    openai_api_key='400568d9a16740b88aff437480544a39',
    temperature=0.2,
    max_tokens=600,
    top_p=1.0
)

history = []
history_entry = 1

def get_history_as_txt():
    txt = ""
    hist = history[::-1]
    
    for i in hist[:3]:
        txt += f"\n<|im_start|>user\n{i['user']}\n<|im_end|>\n"
        txt += f"<|im_start|>assistant\n{i['AI']}\n<|im_end|>"
        
    return txt

def add_to_history(user_msg, ai_msg):
    history.append({'user': user_msg, 'AI': ai_msg})
    
def save_history_to_file():
    global history_entry
    file = open(f"chatbot-cases/chat-{history_entry}.txt", 'w', encoding='utf-8')
    file.write(get_history_as_txt())
    
    file.close()
    
    history_entry += 1
    
def doc_to_str(doc):
    string = ""
    for i in doc:
        string += i.page_content
        
    return string

print(llm2)
doc = open_meteo_docs.OPEN_METEO_DOCS
print(llm2.get_num_tokens(doc))

WARNING! stop is not default parameter.
                    stop was transferred to model_kwargs.
                    Please confirm that stop is what you intended.
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


lc_kwargs={'openai_api_type': 'azure', 'openai_api_base': 'https://openai-nois-intern.openai.azure.com/', 'openai_api_version': '2023-03-15-preview', 'deployment_name': 'gpt-35-turbo', 'openai_api_key': '400568d9a16740b88aff437480544a39', 'temperature': 0, 'max_tokens': 600, 'stop': ['\n', '<|im_end|>', '<|im_sep|>']} verbose=False callbacks=None callback_manager=None client=<class 'openai.api_resources.chat_completion.ChatCompletion'> model_name='gpt-3.5-turbo' temperature=0.0 model_kwargs={'stop': ['\n', '<|im_end|>', '<|im_sep|>']} openai_api_key='400568d9a16740b88aff437480544a39' openai_api_base='https://openai-nois-intern.openai.azure.com/' openai_organization='' openai_proxy='' request_timeout=None max_retries=6 streaming=False n=1 max_tokens=600 deployment_name='gpt-35-turbo' openai_api_type='azure' openai_api_version='2023-03-15-preview'
761


In [7]:
# print(open_meteo_docs.OPEN_METEO_DOCS)
hrm_doc = open("HRM_API_DOC.txt", 'r', encoding='utf-8')
api_doc = hrm_doc.read()
print(api_doc)
print(f"API token length: {count_tokens(api_doc)}")

BASE URL: https://hrm-nois-fake.azurewebsites.net/

HRM system
API DOCUMENTATION
Endpoints:
- /api/User: does not accept any input. Use GET with this endpoint to get a list of users within the HRM system.
- /api/User/me/<id>: accepts an ID string as a path variable. Use GET with this endpoint along with a user's ID to get data of a specific user.
- /api/User/me/<email>: accepts an EMAIL string as a path variable. Use GET with this endpoint along with a user's ID to get data of a specific user.
- /api/User/manager-users: Use GET with this endpoint to get name of a manager user.
- /api/LeaveApplication/<userId>: accepts an ID string as a path variable. Use GET with this endpoint along with a user's ID to get all leave applications submitted by a user.

API token length: 186


In [8]:
chain_new = APIChain.from_llm_and_api_docs(llm2, api_doc, verbose=True)

In [9]:
print(chain_new)

lc_kwargs={'api_request_chain': LLMChain(lc_kwargs={'llm': AzureChatOpenAI(lc_kwargs={'openai_api_type': 'azure', 'openai_api_base': 'https://openai-nois-intern.openai.azure.com/', 'openai_api_version': '2023-03-15-preview', 'deployment_name': 'gpt-35-turbo', 'openai_api_key': '400568d9a16740b88aff437480544a39', 'temperature': 0, 'max_tokens': 600, 'stop': ['\n', '<|im_end|>', '<|im_sep|>']}, verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={'stop': ['\n', '<|im_end|>', '<|im_sep|>']}, openai_api_key='400568d9a16740b88aff437480544a39', openai_api_base='https://openai-nois-intern.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=600, deployment_name='gpt-35-turbo', openai_api_type='azure', openai_api_version='2023-03-15-preview'), 'prompt': PromptTemplate(lc_kwargs={'input_var

In [10]:
chain_new("Users's birthday'?")

Error in on_chain_start callback: 'name'


https://hrm-nois-fake.azurewebsites.net/api/User
{"message":"OK","data":[{"id":"262aa143-307a-4b9f-848d-1e6bf2e5cacb","name":"Hung Bui","email":"hung.bui@nois.vn","fullName":"BÙI TUẤN HƯNG","jobTitle":"Lập trình viên","mobilePhone":"0966772170","birthday":"1997-08-17T00:00:00","offDay":0.5,"yearOffDay":6,"maxDayOff":15,"remainDayOffLastYear":0,"offDayUseRamainDayOffLastYear":1,"gender":1,"personalEmail":"hungbt.dev@gmail.com","avatar":"https://noishrm.blob.core.windows.net/avatar/8fec9283-e82f-43ec-8f61-addb732dabcb.jpeg","acceptOfferDate":"2022-10-31T00:00:00","quitJobDate":"","bonusDayOff":0,"managerId":"","managerFullName":"","departmentId":0,"departmentName":"Development Unit 1","groupUserId":"","groupUserName":"Nhân viên chính thức","hasLeft":"","identityCard":"","dateOfIssue":"2021-08-10T00:00:00","issuedBy":"Cục trưởng CCS QLHC về TTXH","placeOfTemporaryResidence":"502/15/2 Thống Nhất, p.16, Gò Vấp, TP.HCM","placeOfResidence":"161 Đường 458, Ấp Hội Thạnh, Xã Trung An, Huyện Củ C

{'question': "Users's birthday'?",
 'output': "The response contains a list of users within the HRM system. Each user has various details, including their ID, name, email, job title, mobile phone, birthday, and other personal information. However, the specific user's birthday is not provided in the response."}

In [99]:
url = "https://hrm-nois-fake.azurewebsites.net/"
date = str(datetime.now().date())
email = 'nghiem.vo@nois.vn'


def get_users(query: str = None):
    return requests.get(url + '/api/User').json()['data']

def delAll(ID: str):
    lst = []
    get_reply = requests.get(url + f"/api/LeaveApplication/{ID}").json()
    for i in get_reply['data']:
        lst.append(i['id'])
        
    for i in lst:
        del_reply = requests.delete(url + f"/api/LeaveApplication/{i}")
        print(del_reply.text)

def get_user_by_email(email):
    response = requests.get(url + f'/api/User/me?email={email}')
    if response.status_code == 200:
        return response.json()['data']
    
    return f'Error: {response.status_code}'

def get_user_by_id(ID: str):
    reply = requests.get(url + f'/api/User/me?id={ID}')
    if response.status_code != 200:
        return f'Error: {response.status_code}'
    
    if not reply.json()['data']:
        return "Wrong ID, recheck your input."
    
    return reply.json()['data']

def check_manager_name(name):
    response = requests.get(url + '/api/User/manager-users').json()['data']
    
    for data in response:
        if data['fullName'].lower() == name.lower(): 
            return data['id']
        
    return -1
def check_manager_id(managerId):
    response = requests.get(url + '/api/User/manager-users').json()['data']
    
    for data in response:
        if data['id'] == int(managerId): 
            return True
        
    return False
    
def post_method(user_id, manager_id, start_date, end_date):
    start_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    end_datetime = datetime.strptime(end_date, '%Y-%m-%d')
    
    num_days = 0
    
    if end_datetime == start_datetime:
        num_days = 0.5
    else:
        num_days = (end_datetime - start_datetime).days
        
    response = requests.post('https://hrm-nois-fake.azurewebsites.net/api/LeaveApplication/Submit', json = {
        "userId": user_id,
        "reviewUserId": manager_id,
        "relatedUserId": "string",
        "fromDate": start_date,
        "toDate": end_date,
        "leaveApplicationTypeId": 0,
        "leaveApplicationNote": "string",
        "periodType": 0,
        "numberOffDay": num_days
    })
    
    return response
    
lst = []
def submitLeaveApplication(args: str):
    global lst
    lst = args.split(', ')
    
    if len(lst) != 4:
        return "Incorrect number of arguments, this function requires 4 arguments: user's id, manager's id, start date and end date."
    
    
    if '-' not in lst[0]:
        return "Incorrect ID. You can find the correct ID by using the HRM get by email tool."
    
    if check_manager_name(lst[1]) == -1 and check_manager_id(lst[1]) == False:
        return "Ask the user for the correct manager's name. The user either didn't give you a name or gave you the wrong name."
    
    if '-' not in lst[2]:
        return "Ask the user when they want to start their leave. Infer the date based on the user's answer."
    
    if datetime.strptime(lst[2], "%Y-%m-%d") < datetime.strptime(date, "%Y-%m-%d"):
        return "Start date cannot be earlier than current date, ask the user for another date."
    
    if '-' not in lst[3]:
        return "Ask the user when they want to end their leave. Infer the date based on the user's answer."
    
    if datetime.strptime(lst[3], "%Y-%m-%d") < datetime.strptime(lst[2], "%Y-%m-%d"):
        return "End date cannot be earlier than start date, ask the user for another date."
    
    print("\nUserId: ", lst[0])
    print("ReviewerId: ", lst[1])
    print("Start date: ", lst[2])
    print("End date: ", lst[3])
    
    reply = post_method(lst[0], int(lst[1]), lst[2], lst[3])
    
#     return reply.text
    return '{"message": "OK"}'
        
tools = [    
    
    Tool(
        name='HRM get by name',
        func=check_manager_name,
        description='useful for getting the manager\'s id by manager\'s name. Input is a manager\'s name.'
    ),
    
    Tool(
        name='HRM get by email',
        func=get_user_by_email,
        description='useful for getting the user\'s id by user\'s email.'
    ),
    
    Tool(
        name='HRM submit leave',
        func=submitLeaveApplication,
        description=f'''useful for submitting leave applications for a specific user.
Input must include 4 parameters concatenated into a string separated by a comma and space, which are: 
1. user's id: you get from the given email. 
2. the id of manager who approve user's application: firstly, ask user the name of the manager and then find manager's id by name.
3. start date: ask the user when they want to start their leave.
4. end date: ask the user when they want to end their leave.
Check to ensure that 4 parameters are got correctly according to form: user's id, manager's name, start date, end date.

'''
    )
]

# Example: 65255df1-d4c3-479b-a4b8-d63cb3db2beb, LÝ MINH QUÂN, 2023-07-10, 2023-07-11

human = load_tools(['human'])

tools.extend(human)

f'''If you cannot answer by using only 1 tool, try using other provided tools.
If you cannot get any results from tools, say you don't know.
Do not use any other sources other than the ones you obtain from tools.
Suppose the current date is {date} (Year-Month-Day).'''

date = datetime.now().strftime("%Y-%m-%d")
print(date)

prefix = f"""You are an intelligent assistant helping users interact with system called HRM through the use of tools.
Answer in the user's language.
The user chatting with you own the email: {email}
You have access to the following tools:"""

suffix = """

Examples: 

Observation: The user chatting with you own the email nghiem.vo@nois.vn.
Thought: find the user's id by user's email.
Action: HRM get by email
Observation: [65255df1-d4c3-479b-a4b8-d63cb3db2beb] , [manager's id], [start date], [end date]
Thought: Ask the user for the manager's name.
Action: Human
Action Input: 
Thought: find the manager's id by manager's name.
Action: HRM get by name
Observation: [65255df1-d4c3-479b-a4b8-d63cb3db2beb] , [139], [start date], [end date]
Thought: ask user the date of starting leaving (according the format Year-Month-Day)?
Action: Human
Action Input: 
Observation: [65255df1-d4c3-479b-a4b8-d63cb3db2beb] , [139], [2023-07-17], [end date]
Thought: ask user the date of ending leaving (according the format Year-Month-Day)?
Action: Human
Action Input: 
Observation: [65255df1-d4c3-479b-a4b8-d63cb3db2beb] , [139], [2023-07-17], [2023-07-18]
Thought: Got all details for submitting.
Action: HRM submit leave.
Observation: Successfully.

Begin!

{context}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "context", "agent_scratchpad"],
)

history = ConversationBufferWindowMemory(k=3, memory_key="context", human_prefix='User', ai_prefix='Assistant')

llm_chain = LLMChain(llm=llm3, prompt=prompt)

agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True, stop=["\nObservation:", "<|im_end|>", "<|im_sep|>"])
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools, 
    verbose=True,
    handle_parsing_errors="True",
    memory=history
)

2023-07-11


In [100]:
print(prompt.template)

You are an intelligent assistant helping users interact with system called HRM through the use of tools.
Answer in the user's language.
The user chatting with you own the email: nghiem.vo@nois.vn
You have access to the following tools:

HRM get by name: useful for getting the manager's id by manager's name. Input is a manager's name.
HRM get by email: useful for getting the user's id by user's email.
HRM submit leave: useful for submitting leave applications for a specific user.
Input must include 4 parameters concatenated into a string separated by a comma and space, which are: 
1. user's id: you get from the given email. 
2. the id of manager who approve user's application: firstly, ask user the name of the manager and then find manager's id by name.
3. start date: ask the user when they want to start their leave.
4. end date: ask the user when they want to end their leave.
Check to ensure that 4 parameters are got correctly according to form: user's id, manager's name, start date, e

In [101]:
history.clear()
answer = agent_chain.run(input="I'd like to submit a leave application.")

Error in on_chain_start callback: 'name'


Thought: The user wants to submit a leave application.
Action: HRM get by email
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the error persists. (Please include the request ID bd478b96-8510-414b-ad30-8cc3653714a9 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the error persists. (Please include the request ID bd478b96-8510-414b-ad30-8cc3653714a9 in your message.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that! You can retry your request, or contac

I need to provide the HRM get by email action input.
Action: HRM get by email
Action Input: nghiem.vo@nois.vn
Observation: {'id': '65255df1-d4c3-479b-a4b8-d63cb3db2beb', 'name': 'Nghiem Vo', 'email': 'nghiem.vo@nois.vn', 'fullName': 'VÕ NGỌC DUY NGHIÊM', 'jobTitle': 'Lập trình viên', 'mobilePhone': '0843961713', 'birthday': '1999', 'offDay': 3, 'yearOffDay': 0, 'maxDayOff': 10, 'remainDayOffLastYear': 0, 'offDayUseRamainDayOffLastYear': 0, 'gender': 0, 'personalEmail': '', 'avatar': '', 'acceptOfferDate': '', 'quitJobDate': '', 'bonusDayOff': 0, 'managerId': '', 'managerFullName': '', 'departmentId': 0, 'departmentName': 'Development Unit 1', 'groupUserId': '', 'groupUserName': '', 'hasLeft': '', 'identityCard': '', 'dateOfIssue': '', 'issuedBy': '', 'placeOfTemporaryResidence': '', 'placeOfResidence': '', 'relativeName': '', 'relationship': '', 'relativeMobilePhone': '', 'hasBankAccount': '', 'bankName': 'VIB', 'bankBranch': '', 'bankAccountNumber': '863644436', 'birthPlace': 'Los Ang

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the error persists. (Please include the request ID 2849b02a-fee6-4e90-b3f2-ea0216989fe3 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the error persists. (Please include the request ID 2849b02a-fee6-4e90-b3f2-ea0216989fe3 in your message.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that! You can retry your request, or contac

I need to find the manager's id by manager's name.
Action: HRM get by name
Action Input: trần đăng ninh
Observation: 119
Thought:I have retrieved the manager's id. Now I need to ask the user for the start date of their leave.
Observation: Invalid Format: Missing 'Action:' after 'Thought:'
Thought:I need to ask the user for the start date of their leave.
Action: Human
Action Input: When do you want to start your leave? (Please provide the date in the format Year-Month-Day)

When do you want to start your leave? (Please provide the date in the format Year-Month-Day)
2023-07-12

Observation: 2023-07-12
Thought:I have retrieved the start date of the user's leave. Now I need to ask the user for the end date of their leave.
Action: Human
Action Input: When do you want to end your leave? (Please provide the date in the format Year-Month-Day)

When do you want to end your leave? (Please provide the date in the format Year-Month-Day)
2023-07-13

Observation: 2023-07-13
Thought:I have retrieved 

In [36]:
print(answer)
print(lst)

Agent stopped due to iteration limit or time limit.
["[user's id]", "[manager's name]", '[start date]', '[end date]']


In [17]:
delAll("65255df1-d4c3-479b-a4b8-d63cb3db2beb")